NEW CONTINUING: After [finding errors](https://github.com/Data-Science-for-Linguists-2019/Animated-Movie-Gendered-Dialogue/blob/master/code/Disney_code/Messy_Disney_Movies.ipynb) in Frozen and a successful [scraping of Tangled](https://github.com/Data-Science-for-Linguists-2019/Animated-Movie-Gendered-Dialogue/blob/master/code/Disney_code/Scraping_Tangled.ipynb), I've done the same for Frozen.

# Scraping Frozen
Basically, I lost my patience with the Frozen dataframe. I'm doing this myself. (I guess I couldn't *let it go*...sorry, sleep-deprived me lives for puns)

This webpage (link below in code) is actually formatted much more neatly than Tangled's was. Because of this, it was pretty easy to for loop through the html file, make a string out of each line, and go from there!

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import re

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
link = "https://transcripts.fandom.com/wiki/Frozen"

In [6]:
page_response = requests.get(link, timeout=5)

In [7]:
page_content = BeautifulSoup(page_response.content, "html.parser")

In [8]:
#I did this to help find where the script was located, but I can't flash the whole thing
#page_content

In [9]:
script_html = page_content.find("div", {"class": "mw-content-ltr mw-content-text"} ) 
#this portion contains the actual script!!

In [10]:
#I looked at this to see how things were formatted, but I can't show you the whole script
# or else Disney might get angry!
#script_html

In [11]:
#The script is split up at random points, so this is kind of annoying.
# At first I thought that meant I had to for loop through each of these, but that
# actually gave me 5 copies of my script! YIKES!
script_chunks = script_html.find_all("ul")

In [12]:
len(script_chunks)

5

In [13]:
#here's the smallest chunk in the bunch, so you can see what this formatting looks like!
script_chunks[3]

<ul><li> <b>Anna</b>: That's nice. But... 
</li><li> <b>Bulda and Troll#1</b>: So he's a bit of a fixer upper. So he's got a few flaws.
</li><li><b>Troll #4</b>: Like his peculiar brain, dear.
</li><li><b>Troll #5</b>: His thing for the reindeer.
</li><li><b>Troll #4 and Troll #5</b>: That's a little outside of nature's laws.
</li><li><b>Kristoff</b>: This is not about me!
</li><li><b>All Trolls:</b> So he's a bit of a fixer upper, but this we're certain of. You can fix this fixer upper up with a little bit of love. [they push Kristoff towards Anna and they both look embarrassed]
</li><li><b>Anna</b>: Um...
</li><li><b>Kristoff</b>: [to the trolls as they continue to sing] Can we please just stop talking about this? We've got a real, actual problem here.
</li><li><b>Kristoff</b>: Can we please just stop talking about this? We've got a real, actual problem here.
</li><li><b>Bulda</b>: I'll say. So tell me dear, is it the way that he runs scared?
</li><li><b>Troll #5</b>: Or that he's so

### Building a list of tuples
Here, I for loop through each next element in my first chunk (which goes through the whole script), turn each line in the chunk into a string, take out all those html characters, and split on the colons (unlike Tangled, I can do this becauce colons only appear after actual lines!!)

In [14]:
just_lines_list = []
for line in script_chunks[0].next_elements:
    line_str = str(line)
    if line_str.startswith("<li><b>") or line_str.startswith("<li> <b>"): #all lines start with these
        line_str = re.sub(r"<li> ?<b>", '', line_str)
        line_str = line_str.replace("\n</li>", '', 1).replace('"', '').replace("</b>", '')
        line_str_real = re.sub(r"\[[^\]]*\]", '', line_str)
        speaker_line = line_str_real.split(":")
        speaker_line[0] = speaker_line[0].strip().lower()
        speaker_line[1] = speaker_line[1].strip().lower()   
        just_lines_list.append(tuple(speaker_line))

In [15]:
len(just_lines_list)

854

In [16]:
just_lines_list[-1]

('olaf', 'glide and pivot and glide and pivot...')

In [17]:
just_lines_list[0]

('ice harvesters', "born of cold and winter air and mountain rain combining. this icy force both foul and fair has a frozen heart worth mining. so cut through the heart, cold and clear.  strike for love and strike for fear. see the beauty sharp and sheer. split the ice apart and break the frozen heart.  ho! watch your step! let it go! ho! watch your step! let it go! beautiful! powerful! dangerous! cold! ice has a magic can't be controlled. stronger than one, stronger than ten, stronger than a hundred men!  born of cold and winter air and mountain rain combining. this icy force both foul, also this story is about a girl saving another girl from a boy who is very very bad and fair has a frozen heart worth mining.  cut through the heart, cold and clear. strike for love and strike for fear.  there's beauty and there's danger here. split the ice apart! beware the frozen heart.")

### List of tuples created! Time to create a dataframe!!

In [18]:
import pandas as pd

In [19]:
frozen = pd.DataFrame(just_lines_list, columns=["Speaker", "Text"])

In [20]:
frozen.head()

,Speaker,Text
0,ice harvesters,born of cold and winter air and mountain rain ...
1,young kristoff,"come on, sven!"
2,young anna,elsa. psst. elsa! wake up. wake up. wake up. ...
3,young elsa,go play by yourself.
4,young anna,"do you want to build a snowman? come on, come..."


In [21]:
frozen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 2 columns):
Speaker    854 non-null object
Text       854 non-null object
dtypes: object(2)
memory usage: 6.7+ KB


## Sweet, sweet victory!!!!!
For those of you who are curious, the actual webpage has 856 lines...I tried to capture as many as possible, and I'm still not sure where those last two lines got to. But I figure this data is better than what I had, so I think I can live with two missing lines!

Let's pickle it!

In [22]:
frozen.to_pickle((r'..\..\Animated-Movie-Gendered-Dialogue\private\frozen_lines.pkl'))

In [24]:
sorted(frozen.Speaker.unique())

['all boy trolls', 'all girl trolls', 'all trolls', 'anna', 'anna and elsa', 'anna and hans', 'bishop', 'boy', 'boy troll', 'boy troll #1', 'boy troll #2', 'boy troll #3', 'bulda', 'bulda and troll #1', 'bulda and troll#1', 'crowd', 'dock master', 'duke', "duke's guard #1", "duke's guard #2", 'elsa', 'family', 'french dignitary', 'gerda', 'girl troll', 'grand pabbie', 'guard', 'guard #1', 'guard #2', 'guard #3', 'guard #4', 'hans', 'ice harvesters', 'irish dignitary', 'kai', 'kid troll', 'kid troll #1', 'kid troll #2', 'kid troll #3', 'king', 'kristoff', 'man #1', 'man #2', 'man in crowd', 'marshmallow', 'men', 'mother', 'oaken', 'olaf', 'persi', "persi's wife", 'queen', 'spanish dignitary', 'teen elsa', 'townswoman', 'troll', 'troll #1', 'troll #2', 'troll #3', 'troll #4', 'troll #4 and troll #5', 'troll #5', 'troll priest', 'voice of man', 'voice of woman', 'woman', 'woman holding baby', 'woman in crowd', 'woman in crowd #1', 'woman in crowd #2', 'young anna', 'young elsa', 'young el